<a href="https://colab.research.google.com/github/muajnstu/Product-Recommendation-system/blob/main/Refined_Product_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori,association_rules
import zipfile

In [ ]:
order_prod = pd.read_csv('https://media.githubusercontent.com/media/muajnstu/Product-Recommendation-system/refs/heads/main/order_products__prior.csv')
prod = pd.read_csv('https://raw.githubusercontent.com/muajnstu/Product-Recommendation-system/refs/heads/main/products.csv')

In [ ]:
order_prod.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
prod.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
# Merge the DataFrames
df = pd.merge(order_prod, prod, how='inner', on='product_id')

# Count how many times each product was ordered
product_counts = df['product_id'].value_counts()

# Get the top product ID (most frequently ordered)
top_product_id = product_counts.idxmax()

# Filter the DataFrame to only include orders with the top product
df_top_product = df[df['product_id'] == top_product_id]

# First get the order_ids that contain the top product
orders_with_top_product = df[df['product_id'] == top_product_id]['order_id'].unique()

# Then filter the original DataFrame for these order_ids
df_orders_with_top_product = df[df['order_id'].isin(orders_with_top_product)]

In [ ]:
df_orders_with_top_product.shape

(6655436, 7)

In [ ]:
# analyzing only first 10000 data
df = df_orders_with_top_product.head(100000)

In [ ]:
df.shape

(100000, 7)

In [ ]:
prod_cnt = df['product_name'].value_counts()
prod_cnt

product_name
Banana                               7142
Organic Strawberries                  883
Organic Avocado                       811
Organic Baby Spinach                  804
Strawberries                          616
                                     ... 
Crunchy Double Chocolate                1
Hot Thai Red Curry Paste                1
Salted Caramel High Protein             1
Lemon Garlic Pork Loin Filet            1
High pH 9+ Alkaline Ionized Water       1
Name: count, Length: 14128, dtype: int64

In [ ]:
# Most Reordered Products
prod_cnt = df['product_name'].value_counts().reset_index().head(20)
prod_cnt.columns = ['product_name','reorder_count']
prod_cnt

,product_name,reorder_count
0,Banana,7142
1,Organic Strawberries,883
2,Organic Avocado,811
3,Organic Baby Spinach,804
4,Strawberries,616
5,Large Lemon,614
6,Organic Hass Avocado,505
7,Organic Fuji Apple,490
8,Cucumber Kirby,479
9,Organic Whole Milk,462


## **Most Brought Together Products using Apriori Algorithm**

In [ ]:
df_item = df[['order_id','product_name']].copy()
df_item.rename(columns={'order_id':'order','product_name':'items'},inplace=True)
df_item['temp']=1

In [ ]:
# Data Mining
dm = df_item.groupby(['order','items'])['temp'].sum().unstack().fillna(0)

In [ ]:
# Encoder to convert data in form of 0s and 1s
def myencoder(i):
    if i <= 0:
        return 0
    elif i>=1:
        return 1

In [ ]:
dm.map(myencoder)

items,#2 Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,0 Calorie Acai Raspberry Water Beverage,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Greek Yogurt Vanilla,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,...,of Hanover 100 Calorie Pretzels Mini,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,w/Banana Pulp Free Juice,with Crispy Almonds Cereal,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Seasoned Roasted Potatoes Scrambled Eggs & Sausage,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water
order,,,,,,,,,,,,,,,,,,,,,
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# frequent itemsets
freq_itemsets = apriori(dm,min_support=0.01,use_colnames=True).sort_values(by='support', ascending=False)
freq_itemsets

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
6,0.999860,(Banana)
271,0.123618,"(Organic Strawberries, Banana)"
107,0.123618,(Organic Strawberries)
219,0.113538,"(Organic Avocado, Banana)"
55,0.113538,(Organic Avocado)
...,...,...
92,0.010080,(Organic Old Fashioned Rolled Oats)
50,0.010080,(Mango Chunks)
309,0.010080,(Total 2% All Natural Low Fat 2% Milkfat Greek...
313,0.010080,"(Banana, Total 2% Lowfat Plain Greek Yogurt)"


In [ ]:
# Association Rule
rules = association_rules(freq_itemsets,metric='lift',min_threshold=1)
rules.sort_values(by='confidence',ascending=False)

/usr/local/lib/python3.11/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
477,(Total 2% Lowfat Plain Greek Yogurt),(Banana),0.010080,0.99986,0.010080,1.000000,1.00014,1.0,0.000001,inf,0.000141,0.010081,1.000000,0.505041
0,(Organic Strawberries),(Banana),0.123618,0.99986,0.123618,1.000000,1.00014,1.0,0.000017,inf,0.000160,0.123635,1.000000,0.561817
474,(Total 2% All Natural Low Fat 2% Milkfat Greek...,(Banana),0.010080,0.99986,0.010080,1.000000,1.00014,1.0,0.000001,inf,0.000141,0.010081,1.000000,0.505041
2,(Organic Avocado),(Banana),0.113538,0.99986,0.113538,1.000000,1.00014,1.0,0.000016,inf,0.000158,0.113554,1.000000,0.556777
472,(100% Raw Coconut Water),(Banana),0.010080,0.99986,0.010080,1.000000,1.00014,1.0,0.000001,inf,0.000141,0.010081,1.000000,0.505041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,(Banana),(Organic Old Fashioned Rolled Oats),0.999860,0.01008,0.010080,0.010081,1.00014,1.0,0.000001,1.000001,1.000000,0.010081,0.000001,0.505041
471,(Banana),"(Organic Red Radish, Bunch)",0.999860,0.01008,0.010080,0.010081,1.00014,1.0,0.000001,1.000001,1.000000,0.010081,0.000001,0.505041
473,(Banana),(100% Raw Coconut Water),0.999860,0.01008,0.010080,0.010081,1.00014,1.0,0.000001,1.000001,1.000000,0.010081,0.000001,0.505041
475,(Banana),(Total 2% All Natural Low Fat 2% Milkfat Greek...,0.999860,0.01008,0.010080,0.010081,1.00014,1.0,0.000001,1.000001,1.000000,0.010081,0.000001,0.505041


In [ ]:
rules.shape

(478, 14)

In [ ]:
# Putting filter on association rule to find out most brought together item
filtered_rules  = rules[(rules['confidence']>0.16) & (rules['lift']>2)]

In [ ]:
# Create a dictionary to store product associations
product_associations = {}

# Populate the dictionary with filtered rules
for _, rule in filtered_rules.iterrows():
    antecedents = tuple(rule['antecedents'])  # Products in the "if" part of the rule
    consequents = tuple(rule['consequents'])  # Products in the "then" part of the rule
    confidence = rule['confidence']

    # Add the rule to the dictionary
    if antecedents not in product_associations:
        product_associations[antecedents] = []
    product_associations[antecedents].append((consequents, confidence))

In [ ]:
filtered_rules.shape

(9, 14)

# Final Prediction

In [ ]:
def recommend_items(added_product, product_associations):
    recommendations = []

    # Check if the added product is in any association rule
    for antecedents, consequents_list in product_associations.items():
        if added_product in antecedents:
            for consequents, confidence in consequents_list:
                recommendations.extend(consequents)

    # Remove duplicates and the added product itself
    recommendations = list(set(recommendations))
    if added_product in recommendations:
        recommendations.remove(added_product)

    return recommendations

# Example usage
added_product = "Limes"
recommended_items = recommend_items(added_product, product_associations)
print(f"If you add {added_product}, you might also like: {recommended_items}")

If you add Limes, you might also like: ['Banana', 'Large Lemon']


In [ ]:
# Example usage
added_product = "Organic Avocado"
recommended_items = recommend_items(added_product, product_associations)
print(f"If you add {added_product}, you might also like: {recommended_items}")

If you add Organic Avocado, you might also like: []
